<a href="https://colab.research.google.com/github/kay31105/-/blob/main/%E8%82%A1%E7%A5%A8%E8%B3%87%E6%96%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pandas
!pip install tensorflow
!pip install pandas-datareader
!pip install yfinance
!apt install graphviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 61.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      

In [3]:
import pandas as pd
from pandas_datareader import data
import yfinance as yf
import matplotlib.pyplot as plt
yf.pdr_override()
import numpy as np
from scipy import stats
import csv

In [4]:
df=pd.DataFrame()

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


股票代碼處理

In [7]:
all_close_price = {}
start_date = '2021-11-30'
end_date = '2022-11-30'
with open('drive/MyDrive/CSVDATA/aa.csv', newline='', encoding="utf-8", errors='ignore') as csvfile:
  all_stock = csv.DictReader(csvfile)  # 讀取 CSV 檔內容，將每一列轉成一個 dictionary
  for stock in all_stock:
   str_stk_no = str(stock['stk_no']) + ".TW"  # 格式轉換(1234->1234.TW)
   all_close_price[stock['stk_no']] = yf.download(str_stk_no,start=start_date,end=end_date)['Close']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

回歸資料處理

現金股利率(dy)

In [ ]:
dy={}
with open('drive/MyDrive/CSVDATA/dy.csv', newline='', encoding="utf-8", errors='ignore') as csvfile:
  dy = csv.DictReader(csvfile)
  dy_df = pd.DataFrame(dy)
for column in dy_df:
  dy_df[column] = pd.to_numeric(dy_df[column],errors='coerce')
dy_df=dy_df.drop(index=0)
dy_df

In [ ]:
dym=dy_df.mean()
dym

In [ ]:
from google.colab import files
dym.to_csv('./dym.csv',index=False)
files.download('dym.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

EPS

In [ ]:
eps={}
with open('drive/MyDrive/CSVDATA/eps.csv', newline='', encoding="utf-8", errors='ignore') as csvfile:
  eps = csv.DictReader(csvfile)
  eps_df = pd.DataFrame(eps)
for column in eps_df:
  eps_df[column] = pd.to_numeric(eps_df[column],errors='coerce')
eps_df=eps_df.drop(index=0)
eps_df

,,1101 xd,1102 Ȫd,1103 Ūd,1104 d,1108,1109 Hj,1110 Fd,1201,1203,...,9919 d,9921 j,9924 ֿ,9925 sO,9926 s,9927,9928,9929 B,9930 p귽,9931 Y
1,NaN,NaN,4.37,0.40,1.67,0.61,2.41,0.21,1.14,1.89,...,0.51,15.84,3.68,2.50,2.47,4.91,-0.19,0.23,2.83,2.12
2,NaN,NaN,4.26,0.56,1.80,0.63,1.90,0.25,1.16,1.96,...,0.24,16.23,3.44,2.52,2.47,5.12,-0.19,-0.85,2.79,1.96
3,NaN,NaN,4.03,0.72,1.85,0.75,1.42,0.19,1.23,1.92,...,-0.56,16.09,4.24,3.00,2.62,5.09,0.11,-0.77,2.98,2.14
4,NaN,NaN,4.01,-0.17,2.45,1.00,1.76,0.20,1.25,2.26,...,-0.76,17.73,4.96,2.58,2.68,4.60,0.05,-0.67,3.20,2.13


In [ ]:
epsm=dy_df.mean()
epsm

                NaN
1101 xd    4.003956
1102 Ȫd    7.026953
1103 Ūd    4.951083
1104 d     4.834933
             ...   
9927       8.852880
9928       0.000000
9929 B     1.080725
9930 p귽    5.160403
9931 Y     3.457625
Length: 1753, dtype: float64

In [ ]:
from google.colab import files
epsm.to_csv('./epsm.csv',index=False)
files.download('epsm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

本益比(per)

In [ ]:
per={}
with open('drive/MyDrive/CSVDATA/per.csv', newline='', encoding="utf-8", errors='ignore') as csvfile:
  per = csv.DictReader(csvfile)
  per_df = pd.DataFrame(per)
for column in per_df:
  per_df[column] = pd.to_numeric(per_df[column],errors='coerce')
per_df=per_df.drop(index=0)
per_df


,,1101 xd,1102 Ȫd,1103 Ūd,1104 d,1108,1109 Hj,1110 Fd,1201,1203,...,9919 d,9921 j,9924 ֿ,9925 sO,9926 s,9927,9928,9929 B,9930 p귽,9931 Y
1,NaN,14.0776,9.5918,9.3692,11.6928,21.2003,7.4197,104.4542,17.5470,18.5063,...,19.6796,18.8021,11.0144,16.6504,20.6265,9.5435,NaN,28.5147,15.4569,22.0294
2,NaN,14.3507,9.7489,9.5534,11.7472,21.2925,7.4382,105.5283,17.5470,18.2823,...,19.3377,19.9753,11.0010,16.6083,20.6903,8.7800,NaN,28.3030,15.4569,22.3358
3,NaN,14.3962,9.6367,9.4613,11.6928,21.8455,7.4012,108.4820,17.4284,18.1423,...,19.7176,19.8249,10.9742,16.5661,20.6691,8.5891,NaN,28.5147,15.4569,22.3358
4,NaN,14.5327,9.9284,9.4383,11.7472,21.5690,7.4752,107.4079,17.3099,18.0583,...,19.6036,20.6672,11.0278,16.5240,20.8392,8.5554,NaN,28.7566,15.6125,22.2801
5,NaN,14.5631,9.9396,9.5534,11.7472,21.9377,7.6973,108.7505,17.5865,18.2263,...,19.8695,20.7575,11.1352,16.5872,20.9668,8.5779,NaN,28.6961,15.6817,22.3079
6,NaN,14.3967,9.7933,9.3692,11.7472,21.4768,7.4752,106.8709,17.4284,18.3943,...,23.7447,20.3062,11.1352,16.5240,20.9455,8.5218,NaN,29.2706,15.5952,22.1130
7,NaN,14.4422,9.8572,9.3462,11.7200,20.9238,7.4012,106.6023,17.3494,18.4783,...,22.8329,20.4567,11.1352,16.5450,20.7116,8.6340,NaN,28.4844,15.5952,22.1408
8,NaN,14.3815,9.8351,9.2541,11.7744,20.8316,7.3457,105.7968,17.3099,18.3663,...,22.0730,19.0728,11.0815,16.5240,20.7116,8.5330,NaN,28.4240,15.5779,22.2801
9,NaN,14.2146,9.7688,9.1620,11.5840,20.5551,7.3642,102.3060,16.9147,18.6183,...,21.2752,19.0427,11.1352,16.5661,20.6265,8.3758,NaN,28.4542,15.5088,22.0851
10,NaN,14.4270,10.0561,9.3002,11.6112,21.2925,7.4382,103.6486,17.3099,18.3663,...,20.4014,18.5915,11.4035,16.6294,20.6903,8.5667,NaN,28.3635,15.6125,22.2522


In [ ]:
perm=per_df.mean()

In [ ]:
perm

                  NaN
1101 xd     15.788447
1102 Ȫd     10.043567
1103 Ūd     22.452012
1104 d      11.684231
              ...    
9927         8.481855
9928       193.393788
9929 B      27.848220
9930 p귽     16.163737
9931 Y      20.941700
Length: 1753, dtype: float64

In [ ]:
from google.colab import files
perm.to_csv('./perm.csv',index=False)
files.download('perm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

beta跟all_stock_price改INT

In [ ]:
beta={}
with open('drive/MyDrive/CSVDATA/BETA.csv', newline='', encoding="utf-8", errors='ignore') as csvfile:
  beta = csv.DictReader(csvfile)
  beta_df = pd.DataFrame(beta)
for column in beta_df:
  beta_df[column] = pd.to_numeric(beta_df[column],errors='coerce')
beta_df=beta_df.drop(index=0)
beta_df



,stk_no,1101,1102,1103,1104,1108,1109,1110,1201,1203,...,9919,9921,9924,9925,9926,9927,9928,9929,9930,9931
1,NaN,0.4835,0.1836,0.7582,0.5150,0.4797,0.6702,1.1170,-0.0176,0.0274,...,-2.9337,-0.0099,0.3962,0.1397,0.4103,0.0686,0.0481,0.4517,0.1301,0.3746
2,NaN,0.4267,0.2165,0.6151,0.4255,0.4329,0.5730,0.9466,-0.0550,0.0746,...,-2.5584,0.1886,0.3587,0.0478,0.2990,0.0249,-0.0222,0.4194,0.1112,0.2889
3,NaN,0.4055,0.1733,0.6082,0.3824,0.3966,0.5333,1.0133,-0.0682,0.0741,...,-2.4436,0.2475,0.3366,0.0095,0.2842,0.0142,-0.0458,0.3874,0.0909,0.2743
4,NaN,0.3989,0.0549,0.6908,0.5705,0.3171,0.7403,1.2682,0.0664,0.0645,...,-2.8161,-0.0181,0.3348,0.0983,0.3054,0.2125,-0.1970,0.4197,0.1939,0.2282
5,NaN,0.4251,0.0793,0.6485,0.5438,0.2688,0.7112,1.2668,0.0386,0.0516,...,-2.7816,-0.0229,0.3100,0.1156,0.2938,0.2237,-0.2158,0.3458,0.1923,0.2202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,NaN,0.7222,0.3859,0.4998,0.2314,0.6869,0.2876,0.2369,0.1908,0.2740,...,0.7441,1.3749,0.3107,0.2377,-0.0564,0.0181,0.2374,0.6227,0.1967,0.0030
243,NaN,0.6108,0.3787,0.5833,0.2113,0.7050,0.2232,0.2148,0.2571,0.4340,...,0.9439,1.4451,0.2934,0.2729,0.1605,0.0100,-0.7352,0.4073,0.2007,0.1697
244,NaN,0.6168,0.4586,0.5553,0.0741,0.5805,0.2588,0.2358,0.1672,0.4272,...,0.9046,1.3618,0.2497,0.1764,0.0836,-0.0477,0.2286,0.3714,0.1647,0.2605
245,NaN,0.6301,0.4700,0.5625,0.0742,0.5803,0.2555,0.2384,0.1636,0.4468,...,0.9667,1.3224,0.2459,0.1689,0.0827,-0.0519,0.1812,0.3737,0.1553,0.2752


In [ ]:
all_stock_price={}
with open('drive/MyDrive/CSVDATA/all_stock_price.csv', newline='', encoding="utf-8", errors='ignore') as csvfile:
  all_stock_price = csv.DictReader(csvfile)
  df1 = pd.DataFrame(all_stock_price)
  df1 = df1.drop(index=0)

In [ ]:
for column in df0:
  df1[column] = pd.to_numeric(df1[column],errors='coerce')
df1

,stk_no,1101,1102,1103,1104,1108,1109,1110,1201,1203,...,9919,9921,9924,9925,9926,9927,9928,9929,9930,9931
1,NaN,40.8140,38.7874,19.9965,20.0604,10.8203,18.4786,19.4347,21.7479,31.9083,...,24.4153,296.8632,38.5261,37.7919,46.6801,35.6938,15.0341,9.38,42.1130,37.6715
2,NaN,41.3463,39.1106,19.8520,20.4903,10.9153,18.6172,19.5830,21.6504,31.6665,...,24.9066,294.9572,38.5732,37.6017,47.0651,35.6938,15.0341,9.40,42.2551,38.3960
3,NaN,41.0801,39.1568,19.6110,20.3470,10.8678,18.3862,19.4347,21.5529,31.9083,...,25.6434,302.1048,38.5261,37.2214,46.5358,35.6938,14.8853,9.38,42.1604,38.1545
4,NaN,41.1689,39.4801,19.6110,20.5380,10.9153,18.5248,19.2368,21.6504,31.9566,...,25.4469,297.8162,38.6674,37.5542,46.6801,35.6938,14.8853,9.43,42.3499,38.2028
5,NaN,41.9674,39.8495,19.7556,20.4903,10.9153,18.5248,19.4841,21.6992,31.8116,...,25.2504,301.6283,38.6674,37.3165,46.7764,34.6229,15.2574,9.52,42.1604,38.2028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,NaN,33.2000,41.1000,17.4500,21.5500,10.5500,15.5500,17.8500,19.2500,34.2000,...,17.2000,242.0000,41.6500,39.2500,52.0000,36.9000,20.6000,8.64,45.9000,39.2000
244,NaN,33.5500,40.9500,17.3500,21.5500,10.7000,15.8500,17.8000,19.1500,34.2000,...,16.8000,239.5000,41.6000,39.3000,52.0000,36.9500,22.6500,8.49,45.8000,39.2000
245,NaN,33.0000,40.4500,17.2000,21.8000,10.7500,15.6500,17.7000,19.1000,33.6000,...,16.7000,235.0000,41.6500,39.4500,52.0000,37.0000,21.4000,8.48,45.6000,38.4000
246,NaN,33.5500,41.0500,17.4000,21.9000,10.8000,15.6500,17.7500,19.1000,34.2000,...,17.6500,231.0000,41.6500,39.3000,52.0000,37.0000,21.0000,8.50,45.3500,38.8000


In [ ]:
df1

,,1101 xd,1102 Ȫd,1103 Ūd,1104 d,1108,1109 Hj,1110 Fd,1201,1203,...,9919 d,9921 j,9924 ֿ,9925 sO,9926 s,9927,9928,9929 B,9930 p귽,9931 Y
1,2021/11/30,40.814,38.7874,19.9965,20.0604,10.8203,18.4786,19.4347,21.7479,31.9083,...,24.4153,296.8632,38.5261,37.7919,46.6801,35.6938,15.0341,9.38,42.113,37.6715
2,2021/12/1,41.3463,39.1106,19.852,20.4903,10.9153,18.6172,19.583,21.6504,31.6665,...,24.9066,294.9572,38.5732,37.6017,47.0651,35.6938,15.0341,9.4,42.2551,38.396
3,2021/12/2,41.0801,39.1568,19.611,20.347,10.8678,18.3862,19.4347,21.5529,31.9083,...,25.6434,302.1048,38.5261,37.2214,46.5358,35.6938,14.8853,9.38,42.1604,38.1545
4,2021/12/3,41.1689,39.4801,19.611,20.538,10.9153,18.5248,19.2368,21.6504,31.9566,...,25.4469,297.8162,38.6674,37.5542,46.6801,35.6938,14.8853,9.43,42.3499,38.2028
5,2021/12/6,41.9674,39.8495,19.7556,20.4903,10.9153,18.5248,19.4841,21.6992,31.8116,...,25.2504,301.6283,38.6674,37.3165,46.7764,34.6229,15.2574,9.52,42.1604,38.2028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,2022/11/24,33.2,41.1,17.45,21.55,10.55,15.55,17.85,19.25,34.2,...,17.2,242,41.65,39.25,52,36.9,20.6,8.64,45.9,39.2
244,2022/11/25,33.55,40.95,17.35,21.55,10.7,15.85,17.8,19.15,34.2,...,16.8,239.5,41.6,39.3,52,36.95,22.65,8.49,45.8,39.2
245,2022/11/28,33,40.45,17.2,21.8,10.75,15.65,17.7,19.1,33.6,...,16.7,235,41.65,39.45,52,37,21.4,8.48,45.6,38.4
246,2022/11/29,33.55,41.05,17.4,21.9,10.8,15.65,17.75,19.1,34.2,...,17.65,231,41.65,39.3,52,37,21,8.5,45.35,38.8


In [ ]:
df1 = pd.DataFrame(all_close_price)
df1

,1101,1102,1216,1227,1301,1303,1314,1326,1402,1434,...,2892,2912,2915,3008,3034,3042,3045,3105,3231,3481
Date,,,,,,,,,,,,,,,,,,,,,
2021-11-30,41.820908,42.000000,65.400002,51.000000,102.500000,82.500000,12.70,78.099998,27.900000,28.900000,...,22.647058,269.0,100.000000,1990.0,465.5,105.500000,97.000000,NaN,28.600000,19.281767
2021-12-01,42.366398,42.349998,67.099998,51.200001,104.000000,83.500000,12.75,79.400002,28.299999,29.000000,...,23.137255,276.5,100.000000,2000.0,482.5,105.500000,98.300003,NaN,29.299999,19.558010
2021-12-02,42.093655,42.400002,67.699997,51.099998,103.500000,83.800003,12.55,79.699997,28.299999,28.900000,...,23.235294,277.5,97.099998,2015.0,486.0,103.500000,98.800003,NaN,29.150000,19.502762
2021-12-03,42.184570,42.750000,67.500000,51.400002,103.500000,83.599998,12.70,79.300003,28.299999,29.049999,...,23.284313,275.0,97.099998,1995.0,493.0,105.500000,98.300003,NaN,29.150000,19.889502
2021-12-06,43.002804,43.150002,67.300003,51.200001,104.000000,84.199997,12.65,79.699997,28.600000,28.950001,...,23.333332,276.5,96.500000,2030.0,505.0,102.000000,98.000000,NaN,29.299999,20.110497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-23,32.700001,40.750000,64.000000,39.950001,88.900002,76.300003,9.86,72.500000,31.049999,26.200001,...,25.450001,264.5,65.000000,2280.0,291.0,81.199997,92.000000,NaN,27.500000,12.400000
2022-11-24,33.200001,41.099998,65.300003,40.450001,89.699997,77.099998,9.89,73.800003,31.799999,26.400000,...,25.600000,265.0,66.800003,2325.0,302.0,82.500000,93.300003,NaN,27.350000,12.450000
2022-11-25,33.549999,40.950001,65.699997,40.549999,89.900002,77.099998,9.91,74.000000,31.549999,26.350000,...,25.900000,265.0,66.000000,2260.0,294.5,81.599998,94.099998,NaN,27.250000,12.500000


對數收益率

In [ ]:
lr=np.log(df1/df1.shift(1))

In [ ]:
lr.head()

,stk_no,1101,1102,1103,1104,1108,1109,1110,1201,1203,...,9919,9921,9924,9925,9926,9927,9928,9929,9930,9931
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,0.012958,0.008298,-0.007253,0.021204,0.008741,0.007473,0.007602,-0.004493,-0.007607,...,0.019923,-0.006441,0.001222,-0.005046,0.008214,0.000000,0.000000,0.002130,0.003369,0.019049
3,NaN,-0.006459,0.001181,-0.012214,-0.007018,-0.004361,-0.012486,-0.007602,-0.004514,0.007607,...,0.029153,0.023944,-0.001222,-0.010165,-0.011310,0.000000,-0.009947,-0.002130,-0.002244,-0.006310
4,NaN,0.002159,0.008223,0.000000,0.009343,0.004361,0.007510,-0.010235,0.004514,0.001513,...,-0.007692,-0.014297,0.003661,0.008901,0.003096,0.000000,0.000000,0.005316,0.004485,0.001265
5,NaN,0.019210,0.009313,0.007346,-0.002325,0.000000,0.000000,0.012774,0.002251,-0.004548,...,-0.007752,0.012719,0.000000,-0.006350,0.002061,-0.030462,0.024690,0.009499,-0.004485,0.000000


年化波動計算

In [ ]:
lrs=lr.std()

In [ ]:
v=lrs*250*0.05

In [ ]:
v

stk_no         NaN
1101      0.150810
1102      0.128489
1103      0.134292
1104      0.105792
            ...   
9927      0.184890
9928      0.500044
9929      0.224390
9930      0.075682
9931      0.106386
Length: 1759, dtype: float64

In [ ]:
 v.to_csv('./v.csv',index=False)

波動率下載

In [ ]:
from google.colab import files
v.to_csv('./v.csv')
files.download('v.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

全年加總log_return 全年收益

In [ ]:
slr=np.cumsum(lr)

In [ ]:
slr

,stk_no,1101,1102,1103,1104,1108,1109,1110,1201,1203,...,9919,9921,9924,9925,9926,9927,9928,9929,9930,9931
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,0.012958,0.008298,-0.007253,0.021204,0.008741,0.007473,7.601715e-03,-0.004493,-7.606824e-03,...,0.019923,-0.006441,1.221801e-03,-0.005046,8.213800e-03,0.000000,0.000000,2.129926e-03,0.003369,0.019049
3,NaN,0.006499,0.009479,-0.019467,0.014186,0.004380,-0.005013,-4.076600e-17,-0.009007,-7.285839e-17,...,0.049076,0.017503,-1.272853e-16,-0.015211,-3.096041e-03,0.000000,-0.009947,-4.336809e-18,0.001125,0.012740
4,NaN,0.008658,0.017701,-0.019467,0.023529,0.008741,0.002497,-1.023502e-02,-0.004493,1.512568e-03,...,0.041384,0.003205,3.660934e-03,-0.006310,3.469447e-18,0.000000,-0.009947,5.316334e-03,0.005610,0.014005
5,NaN,0.027868,0.027014,-0.012120,0.021204,0.008741,0.002497,2.538620e-03,-0.002242,-3.035161e-03,...,0.033632,0.015924,3.660934e-03,-0.012659,2.060853e-03,-0.030462,0.014744,1.481509e-02,0.001125,0.014005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,NaN,-0.206475,0.057913,-0.136218,0.071628,-0.025298,-0.172553,-8.505662e-02,-0.122006,6.935948e-02,...,-0.350301,-0.204334,7.796543e-02,0.037857,1.079258e-01,0.033235,0.314970,-8.217718e-02,0.086109,0.039773
244,NaN,-0.195988,0.054256,-0.141965,0.071628,-0.011180,-0.153444,-8.786167e-02,-0.127214,6.935948e-02,...,-0.373831,-0.214718,7.676423e-02,0.039130,1.079258e-01,0.034589,0.409839,-9.969076e-02,0.083928,0.039773
245,NaN,-0.212518,0.041971,-0.150648,0.083162,-0.006518,-0.166142,-9.349549e-02,-0.129829,5.165990e-02,...,-0.379801,-0.233686,7.796543e-02,0.042939,1.079258e-01,0.035941,0.353070,-1.008693e-01,0.079551,0.019154
246,NaN,-0.195988,0.056695,-0.139087,0.087739,-0.001878,-0.166142,-9.067461e-02,-0.129829,6.935948e-02,...,-0.324474,-0.250854,7.796543e-02,0.039130,1.079258e-01,0.035941,0.334201,-9.851360e-02,0.074054,0.029516


In [ ]:
slrm=slr.mean()

下載

In [ ]:
slrm.to_csv('./slrm.csv',index=False)

In [ ]:
from google.colab import files
slrm.to_csv('./slrm.csv')
files.download('slrm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

sharpe ratio

In [ ]:
rfr=0

In [ ]:
sharpe_ratio=(lr.mean()-rfr)/lr.std()

In [ ]:
sharpe_ratio

stk_no         NaN
1101     -0.062538
1102      0.024822
1103     -0.045092
1104      0.042142
            ...   
9927      0.012097
9928      0.039149
9929     -0.020451
9930      0.057081
9931      0.017777
Length: 1759, dtype: float64

年化 sharpe ratio

In [ ]:
asr = sharpe_ratio*250**0.5

In [ ]:
asr

stk_no         NaN
1101     -0.988816
1102      0.392474
1103     -0.712963
1104      0.666322
            ...   
9927      0.191269
9928      0.618994
9929     -0.323360
9930      0.902536
9931      0.281075
Length: 1759, dtype: float64

In [ ]:
asr.to_csv('./table.csv',index=False)

In [ ]:
from google.colab import files
asr.to_csv('./asr.csv')
files.download('asr.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Treynor Ratio

In [ ]:
TR=(lr-rfr)/beta_df

In [ ]:
TRM=TR.mean()

In [ ]:
TRM.to_csv('./table.csv',index=False)

In [ ]:
from google.colab import files
TRM.to_csv('./TRM.csv')
files.download('TRM.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

按照產業比較

In [ ]:
esg={}
with open('drive/MyDrive/CSVDATA/esg.csv', newline='', encoding="utf-8", errors='ignore') as csvfile:
  esg = csv.DictReader(csvfile)
  esg_df = pd.DataFrame(esg)
for column in esg_df:
  esg_df[column] = pd.to_numeric(esg_df[column],errors='coerce')
esg_df